In [1]:
import cv2
import numpy as np
import sys; sys.path.append('..')
from scareddataset import iotools
import scareddataset.data_maniputlation as dm
from scareddataset.calibrator import StereoCalibrator
from pathlib import Path

In [2]:
data_dir = Path('/home/dimitrisps/Datasets/Scared_stereo/scared/dataset4/keyframe1')

In [3]:
left =  cv2.imread(str(data_dir/'Left_image.png'))
right = cv2.imread(str(data_dir/'Right_image.png'))
pts3d_left = iotools.load_depthmap_xyz(data_dir/'left_depth_map.tiff')
pts3d_right = iotools.load_depthmap_xyz(data_dir/'right_depth_map.tiff')

depthmap_left = dm.scared_to_depthmap(pts3d_left)
depthmap_right = dm.scared_to_depthmap(pts3d_right)

iotools.save_subpix_png('./depthmap_left.png', depthmap_left)
iotools.save_subpix_png('./depthmap_right.png', depthmap_right)

In [4]:
calibrator = StereoCalibrator()
calib = calibrator.load(str(data_dir/'endoscope_calibration.yaml'))

In [5]:
RT = dm.create_RT(R = calib['R'], T=calib['T'])

In [6]:
right_pts_cp = pts3d_right.copy()
right_to_left = dm.transform_pts(right_pts_cp.reshape(-1,3),np.linalg.inv(RT))

In [7]:
depthmap_right_to_left = dm.pts3d_to_depthmap(right_to_left.reshape(-1,3), calib['K1'], calib['D1'], (1024,1280))[0]

In [8]:
iotools.save_subpix_png('./right_to_left_depthmap.png', depthmap_right_to_left)

In [ ]:
orb = cv2.ORB()

In [ ]:
kp_left, des_left = orb.detectAndCompute(left,None)

In [ ]:
kp_right, des_right = orb.detectAndCompute(right,None)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# create BFMatcher object
bf = cv2.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
# Match descriptors.
matches = bf.match(des_left,des_right)
# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)
# Draw first 10 matches.
img3 = cv2.drawMatches(left,kp_left,right,kp_right,matches[:10],None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img3),plt.show()